# Méthode de plus forte pente

Considérons $f \in C^1$. Le méthode de plus forte pente consiste à calculer itérativement
$$
x_{k+1} = x_k - \alpha^* \nabla f(x^k)
$$
où $\alpha^* \in \arg\min_{\alpha \geq 0} f(x_k - \alpha \nabla f(x_k))$.

In [ ]:
using Optim
using Plots

Nous avons besoin de la librairie `LinearAlgebra` pour accéder à des méthodes comme `det`, qui calcul le déterminant d'une matrice.

In [ ]:
using LinearAlgebra

## Exemple 1

Considérons la fonction bivariée
$$
f(x, y) = 4x^2 - 4xy + 2y^2
$$

In [ ]:
f1(x) = 4x[1]*(x[1]-x[2])+2*x[2]*x[2]

default(size=(600,600), fc=:heat)
x, y = -2.5:0.1:2.5, 0.5:0.1:2.5
z = Surface((x,y)->f1([x,y]), x, y)
surface(x,y,z)

Son gradient est
$$
\nabla f(x, y) = \begin{pmatrix} 8x - 4y \\ 4y - 4x \end{pmatrix}
$$
La matrice hessienne est
$$
\nabla f^2(x,y) =
\begin{pmatrix}
8 & -4 \\ -4 & 4
\end{pmatrix}
$$

In [ ]:
A = [8 -4; -4 4 ]

Les déterminants des mineurs principaux sont

In [ ]:
8
det( A )

Dès lors, la matrice est définie positive. Nous pouvons le confirmer en calculant ses valeurs propres:

In [ ]:
eigvals(A)

Nous calculons le gradient comme

In [ ]:
function f1grad(x)
    return [8*x[1]-4*x[2], 4*x[2]-4*x[1]]
end

Considérons $x_0 = (2, 3)$. Dès lors $\nabla f(x_0) = (4, 4)$.

Nous devons minimiser la fonction univariée
$$
m(\alpha) = f((2, 3) - \alpha(4, 4)) = f(2 - 4\alpha, 3 - 4\alpha)
$$
La dérivée de $m(\alpha)$ est
\begin{align*}
m'(\alpha) &= \nabla_{(x,y)} f(2 - 4\alpha, 3 - 4\alpha)^T \nabla_{\alpha} \begin{pmatrix} 2 - 4\alpha \\ 3 - 4\alpha \end{pmatrix} \\
&= \begin{pmatrix} 8(2-4\alpha) - 4(3-4\alpha) & 4(3-4\alpha) - 4(2-4\alpha)\end{pmatrix}\begin{pmatrix} -4 \\ -4 \end{pmatrix} \\
&= -\begin{pmatrix} 4 - 16\alpha & 4\end{pmatrix}\begin{pmatrix} 4 \\ 4 \end{pmatrix} \\
&= -16+64\alpha-16\\
&= 64\alpha-32
\end{align*}

La dérivée seconde de $m(\alpha)$ est
$$
m''(\alpha) = 64
$$
Le modèle unidimensionel est dès lors strictement convexe. Le minimiseur peut être trouvé en posant $m'(\alpha^*) = 0$, menant à $\alpha^* = \frac{1}{2}$. Ainsi,
$$
x_1 = x_0 - \frac{1}{2}\nabla f(x_0) = (2, 3) - \frac{1}{2}(4, 4) = (0, 1),
$$
et
$$
\nabla f(x_1) = \begin{pmatrix} -4 \\ 4 \end{pmatrix}
$$
La fonction univariée à minimiser est à présent
$$
m(\alpha) = f((0, 1) - \alpha(-4, 4)) = f(4\alpha, 1 - 4\alpha)
$$
et sa dérivée est
\begin{align*}
m'(\alpha) &= \nabla_{(x,y)} f(4\alpha, 1 - 4\alpha)^T \nabla_{\alpha} \begin{pmatrix} 4\alpha \\ 1 - 4\alpha \end{pmatrix} \\
&= ( 8 \times 4\alpha - 4(1-4\alpha), 4(1-4\alpha) - 4\times(4\alpha))\begin{pmatrix} 4 \\ -4 \end{pmatrix} \\
&= ( -4 + 48\alpha, 4 - 32 \alpha)\begin{pmatrix} 4 \\ -4 \end{pmatrix} \\
&= -32+320\alpha
\end{align*}
La racine de $m'(\alpha)$ est $\alpha^* = \frac{1}{10}$, et $m''(\alpha) = 320$, aussi $\alpha^*$ est un minimiseur global.
Nous obtenons
$$
x_2 = \begin{pmatrix} 0 \\ 1 \end{pmatrix} - \frac{1}{10}\begin{pmatrix} -4 \\ 4 \end{pmatrix}
= \begin{pmatrix} \frac{4}{10} \\ \frac{6}{10} \end{pmatrix}
= \begin{pmatrix} \frac{2}{5} \\ \frac{3}{5} \end{pmatrix}
$$
Nous pourrions continuer, mais un tel calcul à la main est fastidieux. Nous allons automatiser la procédure en construisant une fonction Julia.

In [ ]:
function steepestdescent(f::Function, fprime::Function, x0, h::Float64, verbose::Bool = true,
                         record::Bool = false, tol::Float64 = 1e-7, maxiter::Int64 = 1000)

    function fsearch(α::Float64)
        return(f(x-α*grad))
    end

    x = x0
    k = 0

    grad = fprime(x)

    if (verbose || record)
        fx = f(x)
    end
    if (verbose)
        println("$k. x = $x, f($x) = $fx")
    end
    if (record)
        iterates = [ fx x' ]
    end
    
    while ((k < maxiter) && (norm(grad) > tol))
        α = Optim.minimizer(optimize(fsearch, 0, h, GoldenSection()))
        x = x-α*grad
        k += 1
        grad = fprime(x)       
        if (verbose || record)
            fx = f(x)
        end
        if (verbose)
            println("$k. x = $x, f($x) = $fx")
        end
        if (record)
            iterates = [ iterates; fx x' ]
        end
    end

    if (k == maxiter)
        println("WARNING: maximum number of iterations reached")
    end

    if (record)
        return x, iterates
    else
        return x
    end
end

La variante suivante propose d'élargir l'intervalle dans lequel a lieu la recherche unidimensionnelle quand la borne supérieur est atteinte.

Ce n'est valable que pour des fonctions convexes!

L'idée sera généralisée lors de la discussion sur les régions de confiance.

In [ ]:
function steepestdescent_convex(f::Function, fprime::Function, x0, h::Float64, verbose::Bool = true,
        record::Bool = false, tol::Float64 = 1e-7, maxiter::Int64 = 1000)

    function fsearch(α::Float64)
        return(f(x-α*grad))
    end

    x = x0
    k = 0

    grad = fprime(x)

    if (verbose || record)
        fx = f(x)
    end
    if (verbose)
        println("$k. x = $x, f($x) = $fx")
    end
    if (record)
        iterates = [ fx x' ]
    end

    Δ = 1e-6
    
    while ((k < maxiter) && (norm(grad) > tol))
        α = Optim.minimizer(optimize(fsearch, 0, h, GoldenSection()))
        while ((h-α) <= Δ)
            h *= 2
            α = Optim.minimizer(optimize(fsearch, α, h, GoldenSection()))
        end
        h = α
        x = x-α*grad
        k += 1
        grad = fprime(x)       
        if (verbose || record)
            fx = f(x)
        end
        if (verbose)
            println("$k. x = $x, f($x) = $fx")
        end
        if (record)
            iterates = [ iterates; fx x' ]
        end
    end

    if (k == maxiter)
        println("WARNING: maximum number of iterations reached")
    end

    if (record)
        return x, iterates
    else
        return x
    end
end

L'exécution de cette fonction donne

In [ ]:
sol, iter = steepestdescent(f1, f1grad, [2.0,3.0], 2.0, true, true)

In [ ]:
sol, iter = steepestdescent(f1, f1grad, [10.0,10.0], 2.0, true, true)

In [ ]:
sol, iter = steepestdescent(f1, f1grad, [100.0,100.0], 2.0, true, true)

Nous convergeons vers la solution $(0,0)$, mais la méthode est très lente près de la solution.

In [ ]:
sol, iter = steepestdescent(f1, f1grad, [2.0,3.0], 0.1, true, true)

In [ ]:
sol, iter = steepestdescent_convex(f1, f1grad, [2.0,3.0], 0.1, true, true)

In [ ]:
k = [x = i for i=1:length(iter[:,1])]
Plots.plot(k,iter[:,1])

In [ ]:
k = [x = i for i=10:length(iter[:,1])]
Plots.plot(k,iter[10:length(iter[:,1]),1])

## Descente par coordonnée (Coordinate descent)

In [ ]:
function GaussSeidel(f::Function, x0, h::Float64, verbose::Bool = true, δ::Float64 = 1e-6, maxiter::Int64 = 1000)

    function fsearch(α::Float64)
        return(f(x+α*d))
    end

    x = copy(x0)
    n = length(x)
    k = 0
    d = zeros(n)
    
    while true
        x0[:] = x[:]
        k += 1
        
        for i = 1:n
            d[i] = 1.0  # d is now the i^th vector of the canonical basis
            α = Optim.minimizer(optimize(fsearch, -h, h, GoldenSection()))
            x[i] += α
            d[i] = 0.0
        end
        
        if verbose
            println(k, ". ", f(x), " ", x, " ", x0)
        end
        
        if norm(x-x0) < δ
            break
        end
    end
    
    return x
end

In [ ]:
function Jacobi(f::Function, x0, h::Float64, verbose::Bool = true, δ::Float64 = 1e-6, maxiter::Int64 = 1000)

    function fsearch(α::Float64)
        return(f(x0+α*d))
    end

    x = copy(x0)
    n = length(x)
    k = 0
    d = zeros(n)
    α = zeros(n)
    
    while true
        x0[:] = x[:]
        k += 1
        
        for i = 1:n
            d[i] = 1.0  # d is now the i^th vector of the canonical basis
            α[i] = Optim.minimizer(optimize(fsearch, -h, h, GoldenSection()))
            d[i] = 0.0
        end
        x += α
        
        if verbose
            println(k, ". ", f(x), " ", x, " ", x0)
        end
        
        if norm(x-x0) < δ
            break
        end
    end
    
    return x
end

In [ ]:
sol = GaussSeidel(f1, [2.0,3.0], 1.0)

In [ ]:
sol = Jacobi(f1, [2.0,3.0], 1.0)

## Exemple 2

Considérons la fonction bivariée
$$
f(x,y) = \frac{(2-x)^2}{2y^2}+\frac{(3-x)^2}{2y^2} + \ln y
$$
qui est calculée de Julia comme

In [ ]:
f(x) = (2-x[1])*(2-x[1])/(2*x[2]*x[2])+(3-x[1])*(3-x[1])/(2*x[2]*x[2])+log(x[2])

Sa dérivée est
$$
\nabla f(x) =
\begin{pmatrix}
\frac{-2(2-x)}{2y^2}+\frac{-2(3-x)}{2y^2} \\
-\frac{(2-x)^2}{y^3}-\frac{(3-x)^2}{y^3} + \frac{1}{y}
\end{pmatrix} =
\begin{pmatrix}
\frac{x-2}{y^2}+\frac{x-3}{y^2} \\
-\frac{(2-x)^2}{y^3}-\frac{(3-x)^2}{y^3} + \frac{1}{y}
\end{pmatrix}
$$

In [ ]:
function fprime(x)
    return [(x[1]-2)/(x[2]*x[2])+(x[1]-3)/(x[2]*x[2]),
            -(2-x[1])*(2-x[1])/(x[2]*x[2]*x[2])-(3-x[1])*(3-x[1])/(x[2]*x[2]*x[2])+1/x[2]]
end

In [ ]:
default(size=(600,600), fc=:heat)
x, y = -2.5:0.1:2.5, 0.5:0.1:2.5
z = Surface((x,y)->f([x,y]), x, y)
surface(x,y,z, linealpha = 0.3)

In [ ]:
sol = steepestdescent(f, fprime, [1.0,1.0], 2.0)

Le choix de $h$ est important. Considérons par exemple une valeur trop petite: $h = 0.1$.

In [ ]:
sol = steepestdescent(f, fprime, [1.0,1.0], 0.1)

Un trop grand $h$ peut également conduire à des difficultés. Considérons par exemple $h = 10$.

In [ ]:
sol = steepestdescent(f, fprime, [1.0,1.0], 10.0)

Nous devrons nous assurer que les itérés garantissent que $y > 0$ en raison de l'opérateur logarithmique.

Le choix du point de départ est également important afin de s'assurer que l'algorithme converge assez rapidement. Considérons par exemple $x_0 = (0.1, 0.1)$.

In [ ]:
sol = steepestdescent(f, fprime, [0.1,0.1], 2.0)

Maintenant, prenons $x_0 = (100, 100)$.

In [ ]:
sol = steepestdescent(f, fprime, [100.0,100.0], 5.0)

En pratique, nous aurons souvent besoin de connaissances sur la fonction à optimiser afin d'être efficace.

## Fonction de Rosenbrock

$$
f(x,y) = (1-x)^2 + 100(y-x^2)^2
$$

$$
\nabla f(x,y) =
\begin{pmatrix}
-2(1-x)-400x(y-x^2) \\
200(y-x^2)
\end{pmatrix}
$$

$$
\nabla^2 f(x,y) =
\begin{pmatrix}
2 - 400(y-x^2) + 800x^2 & -400x \\
-400x & 200
\end{pmatrix}
=
\begin{pmatrix}
2 - 400y + 1200x^2 & -400x \\
-400x & 200
\end{pmatrix}
$$

In [ ]:
function rosenbrock(x::Vector)
  return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end
 
function rosenbrock_gradient(x::Vector)
  return [-2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1],
          200.0 * (x[2] - x[1]^2)]
end
 
function rosenbrock_hessian(x::Vector)
  h = zeros(2, 2)
  h[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
  h[1, 2] = -400.0 * x[1]
  h[2, 1] = -400.0 * x[1]
  h[2, 2] = 200.0
  return h
end

In [ ]:
default(size=(600,600))
x, y = 0:0.01:1.0, 0:0.01:1.0
z = Surface((x,y)->rosenbrock([x,y]), x, y)
surface(x,y,z, linealpha = 0.3)

In [ ]:
Plots.contour(x,y,z, linealpha = 0.1, levels=2500)

In [ ]:
sol, iter = steepestdescent(rosenbrock, rosenbrock_gradient, [0.0,0.0], 10.0, true, true)

e minimiseur est situé en $(1,1)$. En effet,
$$
\nabla f(1,1) = \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$
et
$$
\nabla^2 f(1,1) =
\begin{pmatrix}
802 & -400 \\ -400 & 200
\end{pmatrix}
$$
Les déterminants des mineurs principaux sont positifs comme ils valent respectivement 802 et $802\times200-400^2= 400$, aussi la matrice hessienne est définie positif.

Cependant, la méthode de plus forte pente converge très lentement.

In [ ]:
plot!(iter[:,2], iter[:,3])

# Minimisation exacte ou approximative?

La minimisation exacte d'une fonction le long de la direction de recherche exige des hypothèses comme l'unimodalité ou la convexité, lesquelles ne sont pas nécessairement satisfaites. Il est plus pratique de miniser la fonction approximativement le long de la direction de recherche au mieux d'une marche arrière (backtracking). Ceci sera fait plus explicitement dans le bloc-notes sur le recherche linéaire.

Pour les fonctions non-convexes, une première approche consiste à fixer la longueur du pas.

In [ ]:
function batchdescent(f::Function, fprime::Function, x0, α::Float64, verbose::Bool = true,
                      record::Bool = false, tol::Float64 = 1e-7, maxiter::Int64 = 1000)

    function fsearch(α::Float64)
        return(f(x-α*grad))
    end

    x = x0
    k = 0

    grad = fprime(x)

    if (verbose || record)
        fx = f(x)
    end
    if (verbose)
        println("$k. x = $x, f($x) = $fx")
    end
    if (record)
        iterates = [ fx x' ]
    end
    
    while ((k < maxiter) && (norm(grad) > tol))
        x = x-α*grad
        k += 1
        grad = fprime(x)       
        if (verbose || record)
            fx = f(x)
        end
        if (verbose)
            println("$k. x = $x, f($x) = $fx")
        end
        if (record)
            iterates = [ iterates; fx x' ]
        end
    end

    if (k == maxiter)
        println("WARNING: maximum number of iterations reached")
    end

    if (record)
        return x, iterates
    else
        return x
    end
end

Nous pouvons être proche de la solution si $\alpha$ est assez petit.

In [ ]:
sol, iter = batchdescent(f1, f1grad, [2.0,3.0], 0.1, true, true)

Mais si $\alpha$ est trop grand, cela ne fonctionne tout simplement pas!

In [ ]:
ol, iter = batchdescent(f1, f1grad, [2.0,3.0], 2.0, true, true)

Si $f \in C^1$, $f$ convexe, et $\nabla f(\cdot)$ est continue au send de Lipschitz, i.e. $\exists L >0$ tel que
$$
\forall x, y,\ \| \nabla f(x) - \nabla f(y) \|_2 \leq L \| x - y\|_2,
$$
nous pouvons retrouver la convergence en considérant une séquence décroissante de longueurs de pas $\alpha_k > 0$ satisfaisant
$$
\sum_{k = 1}^{+\infty} \alpha_k = +\infty,\qquad \sum_{k = 1}^{+\infty} \alpha_k^2 < +\infty.
$$
Exemple: $\alpha_k = \frac{\kappa}{k}$.

In [ ]:
function rbdescent(f::Function, fprime::Function, x0, α0::Float64, verbose::Bool = true,
                   record::Bool = false, tol::Float64 = 1e-7, maxiter::Int64 = 1000)

    function fsearch(α::Float64)
        return(f(x-α*grad))
    end

    x = x0
    k = 0
    α = α0

    grad = fprime(x)

    if (verbose || record)
        fx = f(x)
    end
    if (verbose)
        println("$k. x = $x, f($x) = $fx")
    end
    if (record)
        iterates = [ fx x' ]
    end
    
    while ((k < maxiter) && (norm(grad) > tol))
        k += 1
        α = α0/k 
        x = x-α*grad
        grad = fprime(x)       
        if (verbose || record)
            fx = f(x)
        end
        if (verbose)
            println("$k. x = $x, f($x) = $fx", ", α = ", α)
        end
        if (record)
            iterates = [ iterates; fx x' ]
        end
    end

    if (k == maxiter)
        println("WARNING: maximum number of iterations reached")
    end

    if (record)
        return x, iterates
    else
        return x
    end
end

In [ ]:
ol, iter = rbdescent(f1, f1grad, [2.0,3.0], 2.0, true, true)

In [ ]:
ol, iter = rbdescent(f1, f1grad, [10.0,10.0], 2.0, true, true)

In [ ]:
ol, iter = rbdescent(f1, f1grad, [100.0,100.0], 2.0, true, true)

In [ ]:
ol, iter = rbdescent(f1, f1grad, [100.0,100.0], 0.1, true, true)

Cette technique a été proposée par Robbins et Monro en 1951 dans le contexte de l'approximation stochastique, où la fonction objectif est
$$
f(x) = E[g(x,\xi)]
$$
et à chaque itération, le prochain itéré est calculé comme
$$
x_{k+1} = x_k - \alpha_k \nabla g(x_k,\xi_k)
$$
où $\xi_k$ est tiré de la distribution de $\xi$.

Cette technique, de même que certaines extensions (mini-lots, gradient stochastique moyen, etc.) est toujours très populaire en apprentissage automatique.